In [ ]:
import numpy as np
import pandas as pd
import os, time, re
import pickle, gzip, datetime
from os import listdir, walk
from os.path import isfile, join

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import Grid

In [ ]:
%matplotlib inline

In [ ]:
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, mean_squared_error
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import adjusted_rand_score
import random

In [ ]:
!pip install kshape

In [ ]:
!pip install tslearn

In [ ]:
!pip install hdbscan

In [ ]:
from kshape.core import kshape, zscore
import tslearn
from tslearn.utils import to_time_series_dataset
from tslearn.clustering import KShape
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans
import hdbscan

In [ ]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers import LeakyReLU, Reshape, UpSampling2D, Conv2DTranspose
from keras.layers import BatchNormalization, Input, Lambda
from keras.layers import Embedding, Flatten, dot
from keras import regularizers
from keras.losses import mse, binary_crossentropy
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.optimizers import Adam, RMSprop

In [ ]:
## 로딩한 압축파일 풀기
!unzip -qq "/content/ECG5000.zip"

In [ ]:
## 데이터 로드
# ECG5000 데이터셋에 대한 설명: https://timeseriesclassification.com/description.php?Dataset=ECG5000
data_train = np.loadtxt("ECG5000_TRAIN.txt")
data_test = np.loadtxt("ECG5000_TEST.txt")

In [ ]:
## 요약통계량 확인하기
print("Number of time series:", len(data_train))
print("Number of unique classes:", len(np.unique(data_train[:,0])))
print("Time series length:", len(data_train[0,1:]))

print("Number of time series:", len(data_test))
print("Number of unique classes:", len(np.unique(data_test[:,0])))
print("Time series length:", len(data_test[0,1:]))

In [ ]:
## 클래스 별 샘플 수 확인
print("Number of time series in class 1.0:", 
      len(data_train[data_train[:,0]==1.0]))
print("Number of time series in class 2.0:", 
      len(data_train[data_train[:,0]==2.0]))
print("Number of time series in class 3.0:", 
      len(data_train[data_train[:,0]==3.0]))
print("Number of time series in class 4.0:", 
      len(data_train[data_train[:,0]==4.0]))
print("Number of time series in class 5.0:", 
      len(data_train[data_train[:,0]==5.0]))

In [ ]:
## 각 클래스 별 수치값 보기
for j in np.unique(data_train[:,0]):
    dataPlot = data_train[data_train[:,0]==j]
    cnt = len(dataPlot)
    dataPlot = dataPlot[:,1:].mean(axis=0)
    print(" Class ",j," Count ",cnt)
    plt.plot(dataPlot)
    plt.show()

In [ ]:
# ECG5000 training 데이터셋과 test 데이터셋을 병합한 후, 다시 training 데이터셋과 test 데이터셋으로 나누기 
data_joined = np.concatenate((data_train,data_test),axis=0)
data_train, data_test = train_test_split(data_joined, 
                                    test_size=0.20, random_state=2019)

X_train = to_time_series_dataset(data_train[:, 1:])
y_train = data_train[:, 0].astype(np.int)
X_test = to_time_series_dataset(data_test[:, 1:])
y_test = data_test[:, 0].astype(np.int)

In [ ]:
## 요약통계량 확인하기
print("Number of time series:", len(data_train))
print("Number of unique classes:", len(np.unique(data_train[:,0])))
print("Time series length:", len(data_train[0,1:]))

print("Number of time series:", len(data_test))
print("Number of unique classes:", len(np.unique(data_test[:,0])))
print("Time series length:", len(data_test[0,1:]))

In [ ]:
## 클래스 별 샘플 수 확인
print("Number of time series in class 1.0:", 
      len(data_train[data_train[:,0]==1.0]))
print("Number of time series in class 2.0:", 
      len(data_train[data_train[:,0]==2.0]))
print("Number of time series in class 3.0:", 
      len(data_train[data_train[:,0]==3.0]))
print("Number of time series in class 4.0:", 
      len(data_train[data_train[:,0]==4.0]))
print("Number of time series in class 5.0:", 
      len(data_train[data_train[:,0]==5.0]))

In [ ]:
## 각 클래스 별 수치값 보기
for j in np.unique(data_train[:,0]):
    dataPlot = data_train[data_train[:,0]==j]
    cnt = len(dataPlot)
    dataPlot = dataPlot[:,1:].mean(axis=0)
    print(" Class ",j," Count ",cnt)
    plt.plot(dataPlot)
    plt.show()

In [ ]:
## 데이터 정규화
X_train = TimeSeriesScalerMeanVariance(mu=0., std=1.).fit_transform(X_train)
X_test = TimeSeriesScalerMeanVariance(mu=0., std=1.).fit_transform(X_test)

In [ ]:
### k-shape
## https://tslearn.readthedocs.io/en/stable/gen_modules/clustering/tslearn.clustering.KShape.html
## 논문: http://www1.cs.columbia.edu/~jopa/Papers/PaparrizosSIGMOD2015.pdf
## 논문: http://web5.cs.columbia.edu/~gravano/Papers/2017/tods17.pdf
## 논문: https://fluidsbarrierscns.biomedcentral.com/track/pdf/10.1186/s12987-022-00311-5.pdf
## https://techy8855.tistory.com/m/21
## 성능평가측도: 수정된 Rand 지수 - 우연히 그룹회된 성분에 대해 조정된 두 데이터 군집 간 유사성 측도
# 예측군집과 실제 군집 사이의 군집 할당에서 일치하는 개수를 측정한다.
# 0에 가까운 경우 순전히 무작위로 군집을 할당한 것
# 1에 가까울 경우 예측군집과 실제군집이 정확하게 일치하는 것 

In [ ]:
## k-shape를 사용한 훈련
ks = KShape(n_clusters=5, max_iter=100, n_init=10,verbose=1,random_state=2019)
ks.fit(X_train)

In [ ]:
## 훈련 데이터에 대한 예측값 생성 및 수정된 Rand 지수 계산
preds = ks.predict(X_train)
ars = adjusted_rand_score(data_train[:,0],preds)
print("Adjusted Rand Index on Training Set:", ars)

In [ ]:
## 테스트 데이터에 대한 예측값 생성 및 수정된 Rand 지수 계산
preds = ks.predict(X_train)
preds_test = ks.predict(X_test)
ars = adjusted_rand_score(data_test[:,0],preds_test)
print("Adjusted Rand Index on Test Set:", ars)

In [ ]:
## 군집별 적합도 검정
preds_test = preds_test.reshape(1000,1)
preds_test = np.hstack((preds_test,data_test[:,0].reshape(1000,1)))
preds_test = pd.DataFrame(data=preds_test)
preds_test = preds_test.rename(columns={0: 'prediction', 1: 'actual'})

counter = 0
for i in np.sort(preds_test.prediction.unique()):
    print("Predicted Cluster ", i)
    print(preds_test.actual[preds_test.prediction==i].value_counts())
    print()
    cnt = preds_test.actual[preds_test.prediction==i].value_counts().iloc[1:].sum()
    counter = counter + cnt
print("Count of Non-Primary Points: ", counter)

In [ ]:
## 시계열 k-평균을 사용한 훈련
# https://tslearn.readthedocs.io/en/stable/gen_modules/clustering/tslearn.clustering.TimeSeriesKMeans.html
# TimeSeriesKMeans: metric={"euclidean", "dtw", "softdtw"}, default는 euclidean

km = TimeSeriesKMeans(n_clusters=5, max_iter=100, n_init=100, metric="dtw", verbose=1, random_state=2019)  
km.fit(X_train)

In [ ]:
# 훈련 셋에 대한 예측값 생성 및 수정된 Rand 지수를 사용한 평가
preds = km.predict(X_train)
ars = adjusted_rand_score(data_train[:,0],preds)
print("Adjusted Rand Index of Time Series k-Means on Training Set:", ars)

In [ ]:
# 테스트 셋에 대한 예측값 생성 및 수정된 Rand 지수를 사용한 평가
preds_test = km.predict(X_test)
ars = adjusted_rand_score(data_test[:,0],preds_test)
print("Adjusted Rand Index of Time Series k-Means on Test Set:", ars)